# Packages

In [43]:
import numpy as np
import os
import scipy.io as sio
import glob
from pathlib import Path
import pandas as pd

# Load the data

In [44]:
# you can download the data from the following link
# http://www.cbsr.ia.ac.cn/users/xiangyuzhu/projects/3DDFA/Database/AFLW2000-3D.zip

In [45]:
mylist = [Path(f).stem for f in glob.glob("AFLW2000/*.mat")]
mylist[0]

'image00002'

In [46]:
def get_list_from_filenames(file_path):
    lines=[Path(f).stem for f in glob.glob(file_path)]
    return lines

In [47]:
def get_pt2d_from_mat(mat_path):
    # Get 2D landmarks
    mat = sio.loadmat(mat_path)
    pt2d = mat['pt2d']
    return pt2d

In [48]:
def get_pt3d_68_from_mat(mat_path):
    mat = sio.loadmat(mat_path)
    pt3d = mat['pt3d_68']
    pt3d=pt3d[:2,:]
    X=pt3d.flatten()
    return X

In [49]:
def get_ypr_from_mat(mat_path):
    # Get yaw, pitch, roll from .mat annotation.
    # They are in radians
    mat = sio.loadmat(mat_path)
    # [pitch yaw roll tdx tdy tdz scale_factor]
    pre_pose_params = mat['Pose_Para'][0]
    # Get [pitch, yaw, roll]
    pose_params = pre_pose_params[:3]
    return pose_params

In [50]:
class AFLW2000():
    def __init__(self, data_dir,file_path, img_ext='.jpg', annot_ext='.mat', image_mode='RGB'):
        self.data_dir = data_dir
        self.file_path = file_path
        self.img_ext = img_ext
        self.annot_ext = annot_ext

        filename_list = get_list_from_filenames(data_dir)
        filename_list.sort()
        self.X_train = filename_list
        self.y_train = filename_list ####????????????
        self.image_mode = image_mode
        self.length = len(filename_list)

    def __getitem__(self, index):
        img = Image.open(os.path.join(self.file_path,self.X_train[index] + self.img_ext))
        img = img.convert(self.image_mode)
        mat_path = os.path.join(self.file_path,self.y_train[index] + self.annot_ext)

        # Crop the face loosely
        pt2d = get_pt2d_from_mat(mat_path)

        x_min = min(pt2d[0,:])
        y_min = min(pt2d[1,:])
        x_max = max(pt2d[0,:])
        y_max = max(pt2d[1,:])

        k = 0.20
        x_min -= 2 * k * abs(x_max - x_min)
        y_min -= 2 * k * abs(y_max - y_min)
        x_max += 2 * k * abs(x_max - x_min)
        y_max += 0.6 * k * abs(y_max - y_min)
        img = img.crop((int(x_min), int(y_min), int(x_max), int(y_max)))

        # We get the pose in radians
        pose = get_ypr_from_mat(mat_path)
        # And convert to degrees.
        pitch = pose[0] * 180 / np.pi
        yaw = pose[1] * 180 / np.pi
        roll = pose[2] * 180 / np.pi
        # Bin values
        bins = np.array(range(-99, 102, 3))
        labels =np.digitize([yaw, pitch, roll], bins) - 1
        cont_labels = [yaw, pitch, roll]
          
        #pt_3d_68
        pt_3d=get_pt3d_68_from_mat(mat_path)

        # if self.transform is not None:
        #     img = self.transform(img)
        # return cont_labels
        return img, labels, cont_labels, self.X_train[index],pt_3d

    def __len__(self):
        # 2,000
        return self.length


In [51]:
demo=AFLW2000("AFLW2000/*.mat",'AFLW2000/')

In [52]:
features=[]
labels=[]
imgs=[]
features_68=[]

for i in range(2000):
    features.append(demo.__getitem__(i)[1])
    x = np.array(demo.__getitem__(i)[2], dtype=np.float32)
    labels.append(x)
    imgs.append(demo.__getitem__(i)[3])
    features_68.append(demo.__getitem__(i)[4])

# Generate 2d data frame

In [53]:
points_features=pd.DataFrame(features_68)
labels = np.array(labels, dtype=np.float32)
labels_df=pd.DataFrame(labels,columns=['yaw','pitch','roll'])
pose_data=pd.concat([points_features,labels_df],axis=1)

In [54]:
pose_data.head()

,0,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,yaw,pitch,roll
0,121.868034,122.367607,126.819237,130.831787,137.523132,148.521729,161.528015,182.876678,213.940063,248.005371,...,310.149048,313.233856,315.107117,322.384613,330.965576,330.701965,324.903870,1.044306,-22.874239,4.908885
1,281.238159,277.339417,274.876953,269.523773,257.788269,240.968155,221.801483,205.846298,207.085449,226.185638,...,303.093811,300.724457,300.060974,303.260895,299.485168,300.598602,302.286499,68.155243,26.932743,17.243670
2,236.385101,239.198257,242.566376,243.971375,242.828186,238.567902,232.561859,229.155289,238.303162,259.122467,...,303.122009,302.877289,302.146454,300.489380,303.481873,304.681732,304.631042,50.485413,-10.579652,-13.570645
3,168.029221,177.697510,190.641602,201.395294,211.957214,222.723099,231.350220,244.332855,268.327637,291.832214,...,315.469391,312.701294,310.133301,300.041290,315.544800,319.498596,320.843994,17.143373,-10.048455,-21.392782
4,280.462250,287.249817,293.892456,297.539368,295.677002,287.270355,271.661591,254.701157,250.219299,261.212463,...,306.811768,307.014893,305.351318,309.934814,311.558899,312.920746,312.058350,68.640549,-50.544582,-59.207973


In [55]:
pose_data.to_csv('head_pose.csv',index=False)